## Prototyping for programmatic data discovery from FSDF/ELVIS

A way to pythonically query the ELVIS system without drags and clicks on a web GUI: http://elevation.fsdf.org.au

Currently assembling parts for API calls as defined here:
https://s3-ap-southeast-2.amazonaws.com/elevation.api.doc.fsdf.org.au/index.html#/default/get_list_all_available_data

Ideally working toward direct access to files from S3.


#### Dr Adam Steer
For [ANU Water and Landscape Dynamics](http://wald.anu.edu.au)

Contact: adam.d.steer@gmail.com

## What is the task here?

The Geoscience Australia ELVIS platform allows users to query an API to find out if there are data within a rectangular region of interest (bounding box).

We need to be able to query the API programatically using Python.

The query needs to return enough information to allow decisions on whether data are useful or not, based on whatever metadata we can extract. A list follows:

- available point cloud data
- useful metadata about the available point clouds:
      - collection agency
      - date of collection
      - file size
      - collection method (LiDAR, photogrammetry)

We also need to be able to filter requests on those attributes; such that we can fulfil a request like 'show me LiDAR point clouds collected by the ACT Government after 2015; and tell me how big the download would be'.

## Part 1: what can we get from the API?

In [6]:
import requests
import json

from shapely import geometry
import pyproj

import re

### consistency with an OGC-style bbox
ELVIS uses a [ymin, ymax, xmin, xmax] bounding box declaration. That's OK but inconsistent with common OGC web APIs. 

Task 1 is to use a WCS-style bounding box [xmin, ymin, xmax, ymax] declaration to query the system:

In [7]:
bbox = [148, -36, 149, -35]

In [8]:
ymin = bbox[1]
ymax = bbox[3]
xmin = bbox[0]
xmax = bbox[2]

elvisurl = "https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw"
elvisparams = {
    "ymin": ymin,
    "ymax": ymax,
    "xmin": xmin,
    "xmax": xmax
    }


In [9]:
%%time 
apiresponse = requests.get(elvisurl, params=elvisparams)

CPU times: user 30.5 ms, sys: 5.88 ms, total: 36.4 ms
Wall time: 3.39 s


In [10]:
responsecontent = json.loads(apiresponse.content)

### Parsing the response

The next step is to parse returned JSON into useable sets. For example we might only want to list point clouds, or 1m DEMs, or 5m DEMs, or data from a specific supplier.

It would be very cool to find query parameter filters to add here, eg:

`?datatype="Point Couds"&source="ACT Government"`

...however the FME docs have no clues there yet.

So we grab the whole catalogue dump and filter on the client side:


In [11]:
elvisdata = responsecontent["available_data"]

We can't get straight to the data - the dictionary 'available data' contains a list of 8 elements:

In [12]:
type(responsecontent["available_data"])

list

In [13]:
len(responsecontent["available_data"])

8

Each of the list elements contains a dictionary with the key 'source' and **at least** a value 'jurisdiction'

In [14]:
elvisdata[0]["downloadables"]["Point Clouds"]

{'AHD': [{'index_poly_name': '6786120',
   'file_name': 'Brindabella201212-LID1-C3-AHD_6786120_55_0002_0002.zip',
   'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786120_55_0002_0002.zip',
   'file_size': '89933541',
   'file_last_modified': '20190207',
   'bbox': '148.95163739352972,-35.04703179352147,148.97312091748344,-35.02865307377786'},
  {'index_poly_name': '6786122',
   'file_name': 'Brindabella201212-LID1-C3-AHD_6786122_55_0002_0002.zip',
   'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786122_55_0002_0002.zip',
   'file_size': '86488919',
   'file_last_modified': '20190207',
   'bbox': '148.95120870836485,-35.029007514192884,148.97268781041888,-35.01062897199796'},
  {'index_poly_name': '6786124',
   'file_name': 'Brindabella201212-LID1-C3-AHD_6786124_55_0002_0002.zip',
   'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_67861

#### Create dictionary of sources
...ideally a 'do once' exercise, but it's really fast so we might as well do it every time

This dictionary is fodder for a `sources` filter implemented in `callingelvis` - assuming we have a desired source agency apriori

In [15]:
%%time

i = 0
jurisdictions = {}
while i < len(responsecontent["available_data"]):
    jurisdictions.update( {i : responsecontent["available_data"][i]["source"] } )
    i+=1

CPU times: user 14 µs, sys: 1e+03 ns, total: 15 µs
Wall time: 18.8 µs


In [16]:
jurisdictions

{0: 'NSW Government',
 1: 'ACT Government',
 2: 'QLD Government',
 3: 'TAS Government',
 4: 'SA Government',
 5: 'NT Government',
 6: 'WA Government',
 7: 'Geoscience Australia'}

if there are data, the dictionary has an additional key 'downloadables':

...we'll pocket this for later, when we want to work out how to filter by jurisdiction. Let's move on to see which jurisdictions have point clouds.

In [17]:
elvisdata[0]

{'source': 'NSW Government',
 'downloadables': {'DEMs': {'1 Metre': [{'index_poly_name': '6786120',
     'file_name': 'Brindabella201212-LID1-AHD_6786120_55_0002_0002_1m.zip',
     'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-AHD_6786120_55_0002_0002_1m.zip',
     'file_size': '7208873',
     'file_last_modified': '20190207',
     'bbox': '148.95163739352972,-35.04703179352147,148.97312091748344,-35.02865307377786'},
    {'index_poly_name': '6786122',
     'file_name': 'Brindabella201212-LID1-AHD_6786122_55_0002_0002_1m.zip',
     'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-AHD_6786122_55_0002_0002_1m.zip',
     'file_size': '7263963',
     'file_last_modified': '20190207',
     'bbox': '148.95120870836485,-35.029007514192884,148.97268781041888,-35.01062897199796'},
    {'index_poly_name': '6786124',
     'file_name': 'Brindabella201212-LID1-AHD_6786124_55_0002_0002_1m.zip',
     'file_url': 'htt

In [18]:
if 'downloadables' in elvisdata[0].keys():
    for key in elvisdata[0]:
        print(key)

source
downloadables


So let's check what things are available to download:

In [19]:
for key in elvisdata[0]["downloadables"]:
    print(key)

DEMs
Point Clouds


....and can we get to point clouds yet? nope - need to choose a height reference

In [20]:
for key in elvisdata[0]["downloadables"]["Point Clouds"]:
    print(key)

AHD


Ok what height references are available? hmm. Head down to the section on 'grabbing all the ELVIS metadata' - where the full list of options can be seen, which are:
- AHD
- Ellipsoidal
- Orthometric

...in the data we have in our bbox, we have the options:

In [21]:
for key in elvisdata[7]["downloadables"]["Point Clouds"]:
    print(key)

Orthometric


In [22]:
for dataset in elvisdata:
    if "downloadables" in dataset.keys():
        jurisdictiondata = dataset["downloadables"]
        if "Point Clouds" in jurisdictiondata.keys():
            for key in jurisdictiondata["Point Clouds"]:
                print(key)

AHD
AHD
Ellipsoidal
Orthometric


...so we have all three options!

Finally we're at point cloud metadata!

First, how many do we have?

In [23]:
len(elvisdata[0]["downloadables"]["Point Clouds"]["AHD"])

2405

In [24]:
elvisdata[1]["downloadables"]["Point Clouds"]["AHD"][0:4]

[{'index_poly_name': '6586070',
  'file_name': 'ACT2015_4ppm-C3-AHD_6586070_55_0002_0002.zip',
  'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-AHD_6586070_55_0002_0002.zip',
  'file_size': '196205608',
  'file_last_modified': '20170616',
  'bbox': '148.74207269637532,-35.50100460313498,148.76372090731323,-35.48265938553811'},
 {'index_poly_name': '6606046',
  'file_name': 'ACT2015_4ppm-C3-AHD_6606046_55_0002_0002.zip',
  'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-AHD_6606046_55_0002_0002.zip',
  'file_size': '475693565',
  'file_last_modified': '20170616',
  'bbox': '148.76887110014735,-35.7169778567827,148.79056911937633,-35.698626943144134'},
 {'index_poly_name': '6606048',
  'file_name': 'ACT2015_4ppm-C3-AHD_6606048_55_0002_0002.zip',
  'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-AHD_6606048_55_0002_0002.zip',
  'file_size': '490738661',
  'file_last_

In [25]:
elvisdata[0]["downloadables"]["Point Clouds"]["AHD"][0]["bbox"]

'148.95163739352972,-35.04703179352147,148.97312091748344,-35.02865307377786'

LIDAR or something else?

We know from empirical research that in NSW collections, LiDAR point cloud file names can have `LID` in the filename, and photogrammetric (usually ADS80 stereo products) have `PHO` in the filename. For the ACT, the path to the point cloud may have `Lidar` in its path

### Practical question 1: find me all the LiDAR point clouds in my ROI

For our bbox, how can we find all the LiDAR point clouds?

In [26]:
# for each of the 8 jurisdictions...
for key, value in jurisdictions.items():
    # if there are downloadables...
    if 'downloadables' in elvisdata[key].keys():
        # ...if those downloadables are point clouds
        if "Point Clouds" in elvisdata[key]["downloadables"].keys():
            
            # if the point clouds are LiDAR ones...
            if re.search("-LID|lidar", json.dumps(elvisdata[key]["downloadables"]["Point Clouds"]), re.IGNORECASE):
                # print the jurisdiction
                print(value)
                # print the metadata
                print(elvisdata[key]["downloadables"]["Point Clouds"])
            
            
    

NSW Government
{'AHD': [{'index_poly_name': '6786120', 'file_name': 'Brindabella201212-LID1-C3-AHD_6786120_55_0002_0002.zip', 'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786120_55_0002_0002.zip', 'file_size': '89933541', 'file_last_modified': '20190207', 'bbox': '148.95163739352972,-35.04703179352147,148.97312091748344,-35.02865307377786'}, {'index_poly_name': '6786122', 'file_name': 'Brindabella201212-LID1-C3-AHD_6786122_55_0002_0002.zip', 'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786122_55_0002_0002.zip', 'file_size': '86488919', 'file_last_modified': '20190207', 'bbox': '148.95120870836485,-35.029007514192884,148.97268781041888,-35.01062897199796'}, {'index_poly_name': '6786124', 'file_name': 'Brindabella201212-LID1-C3-AHD_6786124_55_0002_0002.zip', 'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786124_55_0002_0002.zip', 'fi

### Practical question 2: finding LiDAR point clouds from one year

here we need a regexp match on the file name - we can't just find the year anyplace, because the four digit number might be in the long other set of numbers in the filename someplace. So we need a character followed by the year, which is also a bit imprecise:

In [27]:
year = str(2014)

pattern = "[a-zA-Z]" + year
print(pattern)

[a-zA-Z]2014


filenames are a long way down the JSON tree... we need to loop over jurisductions, then later, over elevations, and then over lists of metadata dictionaries in each elevation...

In [28]:
# for each of the 8 jurisdictions...
for key, value in jurisdictions.items():
    # if there are downloadables...
    if 'downloadables' in elvisdata[key].keys():
        # ...if those downloadables are point clouds
        if "Point Clouds" in elvisdata[key]["downloadables"].keys():
            
            # if the point clouds are LiDAR ones...
            if re.search("-LID|lidar", json.dumps(elvisdata[key]["downloadables"]["Point Clouds"]), re.IGNORECASE):
                
                for heightref in elvisdata[key]["downloadables"]["Point Clouds"]:
                    print(heightref)
                    print(len(elvisdata[key]["downloadables"]["Point Clouds"][heightref]))
                    for dataset in elvisdata[key]["downloadables"]["Point Clouds"][heightref]:
                        if re.search(pattern, json.dumps(dataset["file_name"])):
                         #print the jurisdiction
                            print(value)
                        # print the metadata
                            print(dataset["file_name"])
                            print(dataset["file_url"])
                            

AHD
2405
NSW Government
Brindabella201411-LID1-C3-AHD_6506110_55_0002_0002.zip
https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201411-LID1-C3-AHD_6506110_55_0002_0002.zip
NSW Government
Brindabella201411-LID1-C3-AHD_6506112_55_0002_0002.zip
https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201411-LID1-C3-AHD_6506112_55_0002_0002.zip
NSW Government
Brindabella201411-LID1-C3-AHD_6506114_55_0002_0002.zip
https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201411-LID1-C3-AHD_6506114_55_0002_0002.zip
NSW Government
Brindabella201411-LID1-C3-AHD_6526110_55_0002_0002.zip
https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201411-LID1-C3-AHD_6526110_55_0002_0002.zip
NSW Government
Brindabella201411-LID1-C3-AHD_6526112_55_0002_0002.zip
https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201411-LID1-C3-AHD_6526112_55_0002_0002.zip
NSW Government
Brindabella201411-LID1-C3-AHD_6526114_55_0002_0002.zip
https://s3-ap-southeast-2.

In [29]:
if re.search("-LID|lidar", json.dumps(elvisdata[0]["downloadables"]["Point Clouds"]["AHD"][0]["file_url"]), re.IGNORECASE):
    thedict = elvisdata[0]["downloadables"]["Point Clouds"]["AHD"]
else:
    thedict = {"poop": "ok"}

In [30]:
type(thedict)

list

In [31]:
elvisdata[0]["downloadables"]["Point Clouds"]["AHD"][0]["file_url"]

'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786120_55_0002_0002.zip'

In [32]:
thedict

[{'index_poly_name': '6786120',
  'file_name': 'Brindabella201212-LID1-C3-AHD_6786120_55_0002_0002.zip',
  'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786120_55_0002_0002.zip',
  'file_size': '89933541',
  'file_last_modified': '20190207',
  'bbox': '148.95163739352972,-35.04703179352147,148.97312091748344,-35.02865307377786'},
 {'index_poly_name': '6786122',
  'file_name': 'Brindabella201212-LID1-C3-AHD_6786122_55_0002_0002.zip',
  'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786122_55_0002_0002.zip',
  'file_size': '86488919',
  'file_last_modified': '20190207',
  'bbox': '148.95120870836485,-35.029007514192884,148.97268781041888,-35.01062897199796'},
 {'index_poly_name': '6786124',
  'file_name': 'Brindabella201212-LID1-C3-AHD_6786124_55_0002_0002.zip',
  'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786124_55_0002_0002.zip',


In [33]:
pointclouds = responsecontent["available_data"][0]["downloadables"]["Point Clouds"]
print(json.dumps(pointclouds))

{"AHD": [{"index_poly_name": "6786120", "file_name": "Brindabella201212-LID1-C3-AHD_6786120_55_0002_0002.zip", "file_url": "https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786120_55_0002_0002.zip", "file_size": "89933541", "file_last_modified": "20190207", "bbox": "148.95163739352972,-35.04703179352147,148.97312091748344,-35.02865307377786"}, {"index_poly_name": "6786122", "file_name": "Brindabella201212-LID1-C3-AHD_6786122_55_0002_0002.zip", "file_url": "https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786122_55_0002_0002.zip", "file_size": "86488919", "file_last_modified": "20190207", "bbox": "148.95120870836485,-35.029007514192884,148.97268781041888,-35.01062897199796"}, {"index_poly_name": "6786124", "file_name": "Brindabella201212-LID1-C3-AHD_6786124_55_0002_0002.zip", "file_url": "https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201212-LID1-C3-AHD_6786124_55_0002_0002.zip", "file_size": "8890

In [55]:
%%time

result = callelvis([148.77, -35.35, 148.78, -35.34], "lidar", heightref=None)

https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw?ymin=-35.35&ymax=-35.34&xmin=148.77&xmax=148.78
0
NSW Government
NSW Government
-LID|lidar
AHD
1
ACT Government
ACT Government
-LID|lidar
2
QLD Government
QLD Government
-LID|lidar
3
TAS Government
TAS Government
-LID|lidar
4
SA Government
SA Government
-LID|lidar
5
NT Government
NT Government
-LID|lidar
6
WA Government
WA Government
-LID|lidar
7
Geoscience Australia
Geoscience Australia
-LID|lidar
CPU times: user 32.2 ms, sys: 68.5 ms, total: 101 ms
Wall time: 3.14 s


In [54]:
result

{'NSW Government': {'AHD': [{'index_poly_name': '6606086',
    'file_name': 'Brindabella201802-LID2-C3-AHD_6606086_55_0002_0002.zip',
    'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Brindabella201802-LID2-C3-AHD_6606086_55_0002_0002.zip',
    'file_size': '41732593',
    'file_last_modified': '20180625',
    'bbox': '148.76097224996596,-35.35648385610915,148.78257880073627,-35.33813607992712'}]},
 'ACT Government': 'No data',
 'QLD Government': 'No data',
 'TAS Government': 'No data',
 'SA Government': 'No data',
 'NT Government': 'No data',
 'WA Government': 'No data',
 'Geoscience Australia': 'No data'}

In [37]:
def pointcloudfilter(pointtype):

    if pointtype == "lidar":
        pointfilter = "-LID|lidar"
        
    if pointtype == "photo":
        pointfilter = "-PHO"
            
    return pointfilter

def getpointclouds(responsecontent, sourcekey, pointtype, heightref, year):
    """
    function to assemble point cloud URLS into a per-source dictionary
    
    """
    datadict = {}
    heightrefdict = {}
    datalist = []
    
    #print(responsecontent[sourcekey]["downloadables"]["Point Clouds"])
    
    # check if downloadables exist
    #if "downloadables" in responsecontent[sourcekey].keys:
    #    print(sourcekey)
    print(responsecontent[sourcekey]["source"])
        
    #need to decide which point clouds to look for.
    if pointtype is not None:
        pointfilter = pointcloudfilter(pointtype)
        print(pointfilter)
        
        for height in elvisdata[sourcekey]["downloadables"]["Point Clouds"]:
            if heightref is None:
                print(height)
                for dataset in elvisdata[sourcekey]["downloadables"]["Point Clouds"][height]:
                    if re.search(pointfilter, dataset["file_url"], re.IGNORECASE):
                        if year is not None and str(year) in dataset["file_url"]:
                            datalist.append(dataset)
                        elif year is None:
                            datalist.append(dataset)
                    

            elif heightref in height:
                print(height)
                for dataset in elvisdata[sourcekey]["downloadables"]["Point Clouds"][height]:
                    if re.search(pointfilter, dataset["file_url"], re.IGNORECASE):
                        if year is not None and str(year) in dataset["file_url"]:
                            datalist.append(dataset)
                        elif year is None:
                            datalist.append(dataset)
                            
            #if so, add the jurisdiction to the dict
            #datasetdict.update( { "source" : value } )
            #datasetdict.update( { "downloadables" : '' } )

            #print(datasetdict)
        #else:
            #print("no point clouds")
    
    datadict.update({ responsecontent[sourcekey]["source"] : datalist })
    
    #should return one dict per data source
    return datadict



In [38]:
result["NSW Government"]

KeyError: 'NSW Government'

In [ ]:
getpointclouds(elvisdata, 1, "lidar", None, None)

In [ ]:
this = elvisdata[1]["downloadables"]["Point Clouds"].keys()

In [ ]:
list(this)[0]

In [ ]:
dicttest

In [ ]:
dicttest.update({'newkey': [0,1, 2, 3]})

In [ ]:
dicttest

In [ ]:
dicttest.update({'key': [{'newdict': {'param1': 'value', 'param2': 'value'}}]})

In [ ]:
json.dumps(str(dicttest))

In [ ]:
json.loads(str(dicttest))

## Grab all the catalogue
This next step demonstrates pulling a JSON catalogue of all available data in FSDF/ELVIS. This could be used to build an independent catalogue; but requests for actual data would need to run via ELVIS.

In [ ]:
%%time

elvisurl = "https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw"

apiresponse = requests.get(elvisurl)

In [ ]:
allthedata = json.loads(apiresponse.content)

In [ ]:
allthedata

In [ ]:
for key, value in allthedata.items():
    print(key)

In [39]:
for key, value in allthedata.items():
    print(key)

NameError: name 'allthedata' is not defined

In [168]:
allthedata["available_data"][0]["downloadables"]["Point Clouds"]["AHD"][0]

{'index_poly_name': '4886006',
 'file_name': 'Albury201702-LID1-C3-AHD_4886006_55_0002_0002.zip',
 'file_url': 'https://s3-ap-southeast-2.amazonaws.com/nsw.elvis/z55/Albury201702-LID1-C3-AHD_4886006_55_0002_0002.zip',
 'file_size': '101809461',
 'file_last_modified': '20171207',
 'bbox': '146.86670295764924,-36.090549364924414,146.88894445038846,-36.072540408230246'}

In [158]:
for dataset in allthedata["available_data"]:
    if "downloadables" in dataset.keys():
        jurisdictiondata = dataset["downloadables"]
        if "Point Clouds" in jurisdictiondata.keys():
            for key in jurisdictiondata["Point Clouds"]:
                print(key)
                

AHD
AHD
Ellipsoidal
Ellipsoidal
Orthometric


In [ ]:
with open('elviscatalogue.json', 'w') as outfile:
    json.dump(allthedata, outfile)

## Propose a PostGIS structure

#### pointclouds:

| primary key | geometry | collectionmethod | provider | metadata_id | metadata_url | index_poly | file_name | file_url | file_size | last_modified |
|:---:| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | 
| 0 | geom_object | LiDAR or imagery | source | metadata_id | metadata_url | index_poly_name | file_name | file_url | file_size | file_last_modified |
| 1 | bbox | LIDAR | ACTgovernment | null | null | 6606064 | ACT2015_4ppm-C3-ELL_6606064_55_0002_0002.zip | https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-ELL_6606064_55_0002_0002.zip | 648198911 | 20170619 |

#### raster DEMs:

| primary key | geometry | collectionmethod | provider | metadata_id | metadata_url | index_poly | file_name | file_url | file_size | last_modified |
|:---:| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | 

This is more or less the main dynamic query table. Static views can be created for 'production querying'

There is not enough data coming from FSDF's system to create STAC-compliant records